In [1]:
import codecs
import csv
import time
import os
import re
import gzip
import pandas as pd
import numpy as np

In [2]:
PATH_TO_DATA = 'Data/'
PATH_TO_DATA_UK = PATH_TO_DATA+"ukwiki/"

In [3]:
UKWIKI_ART_FNMS = []
for file in os.listdir(PATH_TO_DATA_UK):
    if re.match(r"ukwiki-20180620-pages-meta-current\d{2}-p\d+p\d+.xml_art.csv.gz", file):
        UKWIKI_ART_FNMS.append(file)  

In [4]:
def unpack(file_name):
    file_name_new = file_name.replace(".gz","")
    with gzip.open(file_name, 'rb') as f_in, open(file_name_new, 'wb') as f_out:
        f_out.writelines(f_in)
    return file_name_new

In [5]:
def pack_and_remove(file_name):
    file_name_new = file_name+'.gz'
    with open(file_name, 'rb') as f_in, gzip.open(file_name_new, 'wb') as f_out:
        f_out.writelines(f_in)
    os.remove(file_name)
    return file_name_new

In [6]:
total_links_count = 0
total_red_links_count = 0
df_title_count = None
for fn in UKWIKI_ART_FNMS:
    fn = PATH_TO_DATA_UK+fn
    print(fn)
    fn_new = unpack(fn)
    df_articles = pd.read_csv(fn_new, encoding='UTF-8', quotechar="\"")
    total_links_count += df_articles.shape[0]
    df_red_links = df_articles[df_articles['is_red_link'] == True]
    total_red_links_count += df_red_links.shape[0]
    
    df_title_count_tmp = pd.DataFrame(df_red_links.groupby('link_val').link_val.count())
    df_title_count_tmp.columns = ['link']
    df_title_count_tmp = df_title_count_tmp.reset_index(col_level=1)
    df_title_count_tmp.columns = ['link_title', 'in_count']
    
    if df_title_count is not None:
        df_title_count = df_title_count.append(df_title_count_tmp)        
        #print("append")
    else:
        df_title_count = df_title_count_tmp
        #print("assign")    
    df_title_count = df_title_count.groupby('link_title').in_count.sum().copy()
    df_title_count.columns = ['link']
    df_title_count = df_title_count.reset_index()
    df_title_count.columns = ['link_title', 'in_count']    
    print("total_links_count : {}".format(total_links_count))
    print("total_red_links_count : {}".format(total_red_links_count))
    print("df_title_count size: {}".format(df_title_count.shape))
    os.remove(fn_new)

Data/ukwiki/ukwiki-20180620-pages-meta-current01-p1p5503930.xml_art.csv.gz
total_links_count : 5520671
total_red_links_count : 514801
df_title_count size: (378611, 2)
Data/ukwiki/ukwiki-20180620-pages-meta-current02-p5503931p11007860.xml_art.csv.gz
total_links_count : 11031490
total_red_links_count : 1205062
df_title_count size: (777741, 2)
Data/ukwiki/ukwiki-20180620-pages-meta-current03-p11007861p16511790.xml_art.csv.gz
total_links_count : 16543016
total_red_links_count : 2207420
df_title_count size: (1316184, 2)
Data/ukwiki/ukwiki-20180620-pages-meta-current04-p16511791p22015720.xml_art.csv.gz
total_links_count : 22046931
total_red_links_count : 2838280
df_title_count size: (1553776, 2)


In [7]:
df_title_count = df_title_count.sort_values(['in_count'], ascending=[0])

In [8]:
df_title_count.to_csv(PATH_TO_DATA_UK+'ukwiki-20180620-red_name_count.csv', index = False, encoding='UTF-8', quotechar="\"")

In [9]:
df_title_count

,link_title,in_count
45,(сузір'я),15716
1234002,Сеніор,1660
1453806,ацетилювання,1619
1134964,Подравський регіон‎,1298
1458260,білок розвитку,942
1163735,Південно-Східна Словенія‎,852
1458258,білок внутрішньоклітинного сигналінгу,821
1080011,Обсерваторія Галеакала,786
682869,Генрі Гольт,734
612047,Велика_Британія,731


In [10]:
df_title_count.describe()

,in_count
count,1.553776e+06
mean,1.826698e+00
std,1.388762e+01
min,1.000000e+00
25%,1.000000e+00
50%,1.000000e+00
75%,1.000000e+00
max,1.571600e+04


In [11]:
df_count_by_count = pd.DataFrame(df_title_count.groupby('in_count').in_count.count())

In [12]:
df_count_by_count.columns = ['link']
df_count_by_count = df_count_by_count.reset_index()
df_count_by_count.columns = ['count', 'in_count']  

In [13]:
df_count_by_count = df_count_by_count.sort_values(['count'], ascending=[1])

In [14]:
df_count_by_count.head()

,count,in_count
0,1,1177094
1,2,203920
2,3,68509
3,4,33237
4,5,18475


In [15]:
df_count_by_count.describe()

,count,in_count
count,318.000000,3.180000e+02
mean,249.616352,4.886088e+03
std,893.529776,6.706917e+04
min,1.000000,1.000000e+00
25%,80.250000,1.000000e+00
50%,159.500000,4.000000e+00
75%,248.750000,2.175000e+01
max,15716.000000,1.177094e+06


In [16]:
df_count_by_count.to_csv(PATH_TO_DATA_UK+'ukwiki-20180620-red_count_by_count.csv', index = False, encoding='UTF-8', quotechar="\"")